In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


# Planning a Cool Weekend Trip from Bangalore Using Data

## Setup and Dependencies

In [47]:


# Coordinates for Bangalore, India
origin_lat = 12.9716
origin_lon = 77.5946

# Target radius: 300 km driving distance
max_radius_km = 300

# Number of candidate locations to generate
num_points = 1000

# Approximate expected temperature in Bangalore
bangalore_temp_reference = 33.0  # degrees Celsius


# Standard dependencies
import math
import time
import folium
import spotipy
import requests
import polyline
import numpy as np
import pandas as pd

from datetime import date
from folium.plugins import AntPath
from geopy.distance import geodesic
from collections import defaultdict
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
from spotipy.oauth2 import SpotifyOAuth
from geopy.distance import distance as geo_distance



## Generate Candidate Locations

In [4]:
def generate_spiral_points_km(center_lat, center_lon, max_radius_km=300, num_points=500):
    points = []
    angle = 0
    scale_factor = max_radius_km / (math.sqrt(num_points))  # spread out nicely

    for i in range(num_points):
        radius_km = scale_factor * math.sqrt(i)
        angle += math.pi * (3 - math.sqrt(5))  # golden angle for uniform spacing

        # Convert polar to cartesian
        dx_km = radius_km * math.cos(angle)
        dy_km = radius_km * math.sin(angle)

        # Use geopy to get lat/lon offset from center
        destination = geo_distance(kilometers=math.hypot(dx_km, dy_km)).destination(
            (center_lat, center_lon), math.degrees(math.atan2(dy_km, dx_km))
        )

        points.append((destination.latitude, destination.longitude))

    return points

#  Generate the points from Bangalore
points = generate_spiral_points_km(origin_lat, origin_lon, max_radius_km, num_points)

# Preview first 5 points
print(f" Generated {len(points)} spiral points within {max_radius_km} km of Bangalore")
print("Sample points:")
for pt in points[:5]:
    print(f"   {pt}")


 Generated 1000 spiral points within 300 km of Bangalore
Sample points:
   (12.9716, 77.5946)
   (12.979082403922487, 77.50749358973246)
   (13.045368160343754, 77.69276315946094)
   (12.825340436199092, 77.56823582066993)
   (13.116290646662762, 77.5006840131555)


## Get Weather Data

In [5]:
#  Initialize geocoder
geolocator = Nominatim(user_agent="weather_trip_planner")
geocode_cache = {}  #  cache for reverse geocoding


In [6]:
def get_weather(lat, lon, date="2024-05-24"):
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={date}&end_date={date}"
        "&daily=temperature_2m_max,temperature_2m_min,uv_index_max,weathercode,precipitation_sum,wind_speed_10m_max"
        "&timezone=Asia%2FKolkata"
    )

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        daily = data["daily"]
        return (
            daily["temperature_2m_max"][0],
            daily["temperature_2m_min"][0],
            daily["weathercode"][0],
            daily["precipitation_sum"][0],
            daily["wind_speed_10m_max"][0],
            daily["uv_index_max"][0]
        )

    except Exception as e:
        print(f"Error fetching weather for ({lat}, {lon}): {e}")
        return None, None, None, None, None, None


In [7]:
def reverse_geocode(lat, lon):
    key = (round(lat, 3), round(lon, 3))  #  avoid duplicates
    if key in geocode_cache:
        return geocode_cache[key]

    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language='en')
        name = location.address.split(",")[0] if location else "Unknown"
    except:
        name = "Unknown"

    geocode_cache[key] = name
    return name


In [8]:
weather_data = []

for idx, (lat, lon) in enumerate(points):
    max_temp, min_temp, weather_code, precipitation, wind_kmph, uv_index = get_weather(lat, lon)

    if max_temp is not None:
        location_name = reverse_geocode(lat, lon)

        weather_data.append({
            "Latitude": lat,
            "Longitude": lon,
            "Location": location_name,
            "Max Temp (°C)": max_temp,
            "Min Temp (°C)": min_temp,
            "Weather Code": weather_code,
            "Precipitation (mm)": precipitation,
            "Wind (km/h)": wind_kmph,
            "UV Index": uv_index
        })

        print(f" {idx+1}/{len(points)} fetched: {location_name} — {max_temp}°C, Wind: {wind_kmph} km/h, UV: {uv_index}")

    else:
        print(f" {idx+1}/{len(points)} skipped: failed to fetch weather")

    time.sleep(1) 


 1/1000 fetched: St. Joseph's Indian High School — 29.8°C, Wind: 16.0 km/h, UV: None
 2/1000 fetched: Srigandhakavalu — 30.2°C, Wind: 13.3 km/h, UV: None
 3/1000 fetched: Maragondanahalli — 30.7°C, Wind: 16.3 km/h, UV: None
 4/1000 fetched: Bilvaradhahalli — 30.0°C, Wind: 12.8 km/h, UV: None
 5/1000 fetched: Kasaghattapura — 30.9°C, Wind: 16.1 km/h, UV: None
 6/1000 fetched: Jantagondahalli — 30.8°C, Wind: 15.5 km/h, UV: None
 7/1000 fetched: Koluru — 30.9°C, Wind: 12.7 km/h, UV: None
 8/1000 fetched: Mutthugadhahalli — 31.1°C, Wind: 15.0 km/h, UV: None
 9/1000 fetched: Doddahegade — 30.5°C, Wind: 15.8 km/h, UV: None
 10/1000 fetched: Srinivasapura — 31.1°C, Wind: 17.9 km/h, UV: None
 11/1000 fetched: Belamangala — 31.4°C, Wind: 16.3 km/h, UV: None
 12/1000 fetched: Allalasandra — 31.1°C, Wind: 12.6 km/h, UV: None
 13/1000 fetched: Chikka Tumakuru — 30.6°C, Wind: 18.3 km/h, UV: None
 14/1000 fetched: Kalasthipuram — 31.8°C, Wind: 19.4 km/h, UV: None
 15/1000 fetched: Polohalli — 30.4°C

In [48]:
df = pd.DataFrame(weather_data)
print(f"DataFrame created with {len(df)} rows.")
df.head()

DataFrame created with 10 rows.


,Latitude,Longitude,Location,Max Temp (°C),Min Temp (°C),Precipitation (mm),Wind (km/h),UV Index,Weather Code
0,12.97185,77.59470,Vittal Mallya Road,29.8,21.7,3.5,16.0,None,55
1,12.91002,77.48120,Bangalore-Mysore Road,31.0,22.3,2.9,12.6,None,53
2,12.73739,77.24689,Bengaluru-Mysuru Expressway,30.8,22.9,7.1,16.8,None,63
3,12.54619,76.92915,Bengaluru-Mysuru Expressway,31.9,22.9,16.5,15.8,None,65
4,12.35963,76.66202,Bangalore - Mysore Expressway,30.7,22.2,1.2,19.9,None,51


## Create Weather DataFrame

In [10]:
# Filter for places with Max Temp below 25°C
cool_places = df[df["Max Temp (°C)"] < 25].reset_index(drop=True)

print(f" Found {len(cool_places)} cool places with temp < 25°C")
cool_places[["Location", "Max Temp (°C)", "Precipitation (mm)", "Wind (km/h)"]]


 Found 24 cool places with temp < 25°C


,Location,Max Temp (°C),Precipitation (mm),Wind (km/h)
0,Thalavadi,24.4,5.6,14.0
1,Bandipura,24.9,10.6,11.1
2,Kadasholai,22.2,14.3,9.6
3,Benthatty,24.7,10.4,7.9
4,Gudalur,24.4,14.1,9.8
5,Coonoor - Kattabettu Road,18.6,9.9,12.8
6,Pykara,17.7,9.0,9.4
7,Gudalur - Sulthan Bathery Road,23.6,18.2,7.9
8,Kundah,18.0,10.7,12.9
9,Gudalur,23.6,14.6,4.8


In [11]:
cool_places_sorted = cool_places.sort_values(by="Max Temp (°C)", ascending=False).reset_index(drop=True)

cool_places_sorted[["Location", "Max Temp (°C)", "Precipitation (mm)", "Wind (km/h)"]].head(19)


,Location,Max Temp (°C),Precipitation (mm),Wind (km/h)
0,Bandipura,24.9,10.6,11.1
1,Nilambur,24.8,37.5,5.6
2,Benthatty,24.7,10.4,7.9
3,Thalavadi,24.4,5.6,14.0
4,Gudalur,24.4,14.1,9.8
5,Madikeri taluku,24.3,13.9,11.3
6,Bhagamandala,24.1,7.1,10.2
7,Vadakara,23.7,18.6,10.5
8,Valparai,23.7,28.2,11.6
9,Mukkali,23.6,32.6,7.7


In [12]:
moderate_rain_spots = cool_places_sorted[
    (cool_places_sorted["Precipitation (mm)"] > 0) &
    (cool_places_sorted["Precipitation (mm)"] <= 15)
].reset_index(drop=True)

print(f" Found {len(moderate_rain_spots)} cool places with moderate rain.")


 Found 11 cool places with moderate rain.


In [13]:
moderate_rain_spots[["Location", "Max Temp (°C)", "Precipitation (mm)", "Wind (km/h)"]]


,Location,Max Temp (°C),Precipitation (mm),Wind (km/h)
0,Bandipura,24.9,10.6,11.1
1,Benthatty,24.7,10.4,7.9
2,Thalavadi,24.4,5.6,14.0
3,Gudalur,24.4,14.1,9.8
4,Madikeri taluku,24.3,13.9,11.3
5,Bhagamandala,24.1,7.1,10.2
6,Gudalur,23.6,14.6,4.8
7,Kadasholai,22.2,14.3,9.6
8,Coonoor - Kattabettu Road,18.6,9.9,12.8
9,Kundah,18.0,10.7,12.9


## Find Tourist Attractions Nearby

In [14]:
opentrip_api_key = "ENTER-YOUR-API-KEY"


In [15]:
def get_nearby_attractions(lat, lon, radius=10000, limit=5):
    try:
        # Get list of places within radius
        places_url = (
            f"https://api.opentripmap.com/0.1/en/places/radius?"
            f"radius={radius}&lon={lon}&lat={lat}"
            f"&rate=2&format=json&limit={limit}&apikey={opentrip_api_key}"
        )
        response = requests.get(places_url)
        response.raise_for_status()
        places_data = response.json()

        # Fetch names of each place
        names = [place["name"] for place in places_data if place.get("name")]
        return ", ".join(names) if names else "No major attractions nearby"

    except Exception as e:
        print(f" Error fetching attractions for ({lat}, {lon}): {e}")
        return "Attractions unavailable"


In [49]:
attractions = []

for idx, row in moderate_rain_spots.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    nearby = get_nearby_attractions(lat, lon)
    attractions.append(nearby)
    print(f"{idx+1}/{len(moderate_rain_spots)} — {row['Location']} → {nearby}")
    time.sleep(1)  

moderate_rain_spots["Nearby Attractions"] = attractions


1/11 — Bandipura → No major attractions nearby
2/11 — Benthatty → No major attractions nearby
3/11 — Thalavadi → No major attractions nearby
4/11 — Gudalur → Mudumalai National Park/Tiger Reserve, Nilgiri Biosphere Reserve
5/11 — Madikeri taluku → Abbey Falls, Pushpagiri WLS
6/11 — Bhagamandala → Brahmagiri, Talakaveri WLS, Devaragundi
7/11 — Gudalur → No major attractions nearby
8/11 — Kadasholai → Kodanadu View Point
9/11 — Coonoor - Kattabettu Road → Sim's Park, Coonoor, Catherine Falls - Geddhehaada Halla, Wellington, Dolphin's nose, Nilgiri Mountain Railway
10/11 — Kundah → Katary Falls, Lovedale, Deer Park, Ooty, Ketti Mountain Railway Staion, ooty
11/11 — Pykara → Pykara reservoir


In [50]:
#  Filter for locations with real attractions
has_attractions = moderate_rain_spots[
    moderate_rain_spots["Nearby Attractions"] != "No major attractions nearby"
].reset_index(drop=True)

print(f" Found {len(has_attractions)} places with real attractions.\n")

#  Print all location names with their attractions
for idx, row in has_attractions.iterrows():
    print(f"{idx+1}. {row['Location']} → {row['Nearby Attractions']}")


 Found 7 places with real attractions.

1. Gudalur → Mudumalai National Park/Tiger Reserve, Nilgiri Biosphere Reserve
2. Madikeri taluku → Abbey Falls, Pushpagiri WLS
3. Bhagamandala → Brahmagiri, Talakaveri WLS, Devaragundi
4. Kadasholai → Kodanadu View Point
5. Coonoor - Kattabettu Road → Sim's Park, Coonoor, Catherine Falls - Geddhehaada Halla, Wellington, Dolphin's nose, Nilgiri Mountain Railway
6. Kundah → Katary Falls, Lovedale, Deer Park, Ooty, Ketti Mountain Railway Staion, ooty
7. Pykara → Pykara reservoir


In [18]:
pip install polyline

In [19]:
ors_key = "5b3ce3597851110001cf6248185acb16f02148c69e3283a4451dbf58"
coordinates = [[77.5946, 12.9716], [75.7394, 12.4306]]  # [lon, lat] for ORS

url = "https://api.openrouteservice.org/v2/directions/driving-car"
headers = {
    "Authorization": ors_key,
    "Content-Type": "application/json"
}

body = {
    "coordinates": coordinates,
    "instructions": False,
    "geometry": True
}

response = requests.post(url, json=body, headers=headers)

print("Status Code:", response.status_code)
print("Raw Response:", response.text)

# Only parse JSON if response is OK
if response.status_code == 200:
    route = response.json()
else:
    print("Failed to fetch route. Check API key, URL, or quota.")
    route = None


Status Code: 200
Raw Response: {"bbox":[75.739182,12.30819,77.594944,12.972022],"routes":[{"summary":{"distance":253741.8,"duration":11479.6},"bbox":[75.739182,12.30819,77.594944,12.972022],"geometry":"aqdnA{erxMa@xAFXHHJJNDHCpEk@tIkAxAOnDg@`AKb@\\HV?LuAdBYd@Uf@I^c@|CUbCCn@JjBl@`DBHLlADd@JzA@NFn@P`Ab@bA`B~AfBvAbCpB`@V\\V~CnClBbBZl@Lh@Fz@RhBHt@PvBLtBJjAAfAAXW|EOdDApA@dDIhAG\\q@`CMj@SlAWbAkApDI^Ir@@d@Fv@H^hA~ZLfAP|@^xAXhAtAjEP`@z@~ClBrJfBvHd@bCF^LxHFbAx@fF\\tC^tBl@tAV^FJRXPXh@z@lAbCv@r@|CfBd@d@^d@Vf@Pt@bAhEd@vBl@zCDPrApGTrD`@~DFb@Db@@LFh@JlAV|BTz@b@dAPd@lA`Cn@`Ad@t@~CrDHJfA|AZz@Jf@DNFXThAl@jAp@|@R\\dAfA`@b@\\d@jBvDXb@xAfCJLR`@Ph@RdAFRTj@JPz@dAxAfBp@v@~@z@p@t@`@l@PXZh@n@rAFNd@n@h@lAb@~@^hARz@Ll@Jh@Fd@nAxFdAhDvCrId@bA`AhBh@t@r@x@fC`C|ErDPJ~@l@NJv@f@hFlDh@ZvAz@l@d@p@h@\\Z~@r@rBnBDF|@bBd@v@BBPVl@`ApBtBxAvAR`@\\l@vAhHL`BBxC@t@@|AHlA^tF\\jCXtAPz@ThAHl@BL?BDR?@d@bCh@tBN^Zv@pD~EzCzDpBvBpFlJf@n@NLpAj@~BdAr@f@NVXh@JZLXd@tBj@rB`@j@n@p@nC`CtAdBzB`DhBzCFJV^xCrD~A|ALNvB`CvBrBt@x@^j@RZl@zBd@zADJxAhDj@p

In [52]:
# GraphHopper API setup
gh_key = "ENTER-YOUR-API-KEY"  
url = "https://graphhopper.com/api/1/route"

# Start: Bangalore (77.5946, 12.9716), End: Madikeri (75.7394, 12.4306)
coordinates = [[77.5946, 12.9716], [75.7394, 12.4306]]  

# Make request to GraphHopper
params = {
    "point": [f"{lat},{lon}" for lon, lat in coordinates],  # convert to lat,lon strings
    "vehicle": "car",
    "points_encoded": False,
    "key": gh_key
}

response = requests.get(url, params=params)
if response.status_code == 200:
    print(" Route fetched successfully!")
    route_data = response.json()
    coordinates = route_data["paths"][0]["points"]["coordinates"]

    # Convert [lon, lat] → (lat, lon)
    decoded_points = [(lat, lon) for lon, lat in coordinates]
    print(f"Total route points: {len(decoded_points)}")

    # Sample 10 evenly spaced points from route
    sampled_points = [decoded_points[i] for i in np.linspace(0, len(decoded_points)-1, 10, dtype=int)]

    # Display sampled points
    for i, pt in enumerate(sampled_points):
        print(f" Point {i+1}: {pt}")

else:
    print(f" Failed to fetch route. Status Code: {response.status_code}")
    print("Raw Response:", response.text)


 Route fetched successfully!
Total route points: 2582
 Point 1: (12.971656, 77.594714)
 Point 2: (12.887944, 77.48494)
 Point 3: (12.739835, 77.290311)
 Point 4: (12.568842, 77.003042)
 Point 5: (12.436932, 76.711279)
 Point 6: (12.355523, 76.523615)
 Point 7: (12.338631, 76.19283)
 Point 8: (12.445286, 75.926988)
 Point 9: (12.436019, 75.80275)
 Point 10: (12.430586, 75.739318)


In [21]:
encoded = route["routes"][0]["geometry"]
decoded_points = polyline.decode(encoded)
print(f"Total route points: {len(decoded_points)}")


Total route points: 2562


In [22]:
sampled_points = [decoded_points[i] for i in np.linspace(0, len(decoded_points)-1, 10, dtype=int)]
for i, pt in enumerate(sampled_points):
    print(f"Point {i+1}: {pt}")


Point 1: (12.97185, 77.5947)
Point 2: (12.91002, 77.4812)
Point 3: (12.73739, 77.24689)
Point 4: (12.54619, 76.92915)
Point 5: (12.35963, 76.66202)
Point 6: (12.33478, 76.49222)
Point 7: (12.32986, 76.20724)
Point 8: (12.45748, 75.96035)
Point 9: (12.43898, 75.80922)
Point 10: (12.43059, 75.73932)


## Get Weather Along Final Route

In [54]:
# Initialize geolocator
geolocator = Nominatim(user_agent="weather-playlist-planner")

# Function to fetch past weather using Open-Meteo
def get_weather(lat, lon, date="2024-05-24"):
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}&start_date={date}&end_date={date}"
        "&daily=temperature_2m_max,temperature_2m_min,uv_index_max,"
        "weathercode,precipitation_sum,wind_speed_10m_max"
        "&timezone=Asia%2FKolkata"
    )
    try:
        response = requests.get(url)
        response.raise_for_status()
        daily = response.json()["daily"]
        return {
            "Max Temp (°C)": daily["temperature_2m_max"][0],
            "Min Temp (°C)": daily["temperature_2m_min"][0],
            "Precipitation (mm)": daily["precipitation_sum"][0],
            "Wind (km/h)": daily["wind_speed_10m_max"][0],
            "UV Index": daily["uv_index_max"][0],
            "Weather Code": daily["weathercode"][0]
        }
    except Exception as e:
        print(f"Error fetching weather for ({lat}, {lon}):", e)
        return None

# Function to get location name using reverse geocoding
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en')
        return location.address.split(",")[0]
    except:
        return "Unknown"

# Final list to build the DataFrame
weather_data = []

for idx, (lat, lon) in enumerate(sampled_points):
    print(f"Processing point {idx+1}/{len(sampled_points)}")

    weather = get_weather(lat, lon)
    location_name = reverse_geocode(lat, lon)

    if weather:
        row = {
            "Latitude": lat,
            "Longitude": lon,
            "Location": location_name
        }
        row.update(weather)
        weather_data.append(row)

    time.sleep(1)  # prevent hitting API rate limits

# Create DataFrame
df_weather_route = pd.DataFrame(weather_data)
print(" df_weather_route created!")
df_weather_route.head(10)


Processing point 1/10
Processing point 2/10
Processing point 3/10
Processing point 4/10
Processing point 5/10
Processing point 6/10
Processing point 7/10
Processing point 8/10
Processing point 9/10
Processing point 10/10
 df_weather_route created!


,Latitude,Longitude,Location,Max Temp (°C),Min Temp (°C),Precipitation (mm),Wind (km/h),UV Index,Weather Code
0,12.971656,77.594714,St. Joseph's Indian High School,29.8,21.7,3.5,16.0,None,55
1,12.887944,77.484940,NICE Peripheral Ring Road,31.1,22.4,2.9,12.6,None,53
2,12.739835,77.290311,Bengaluru-Mysuru Expressway,30.9,23.0,7.1,16.8,None,63
3,12.568842,77.003042,Bengaluru-Mysuru Expressway,31.8,23.3,10.4,12.9,None,63
4,12.436932,76.711279,Bangalore-Mysore Road,31.3,22.7,2.0,18.3,None,53
5,12.355523,76.523615,NH275,29.3,21.3,3.7,16.3,None,55
6,12.338631,76.192830,NH275,27.9,20.8,7.0,11.9,None,63
7,12.445286,75.926988,NH275,28.3,21.4,4.4,17.3,None,61
8,12.436019,75.802750,Mysore Road,25.3,20.0,7.2,14.8,None,63
9,12.430586,75.739318,Ganapathi Street,23.8,19.4,6.4,13.2,None,63


## Map Weather to Playlist Mood

In [26]:
mood_count = defaultdict(int)
used_queries = set()  # Tracks Spotify query strings already used

# Backup combos when all variations are exhausted
fallback_moods = [
    (" Fusion Drive", "Fusion Hindi Tamil"),
    (" Open Highway", "Hindi Roadtrip Classics"),
    (" Soulful Silence", "Minimal Ambient"),
    (" Retro Rewind", "Old Kannada"),
    (" Starry Chill", "English Lofi"),
    (" Melodic techno", "Electronic + Chillstep"),
    (" Calm Commute", "Instrumental Focus"),
]

def map_weather_to_playlist(temp, rain, wind, uv, location):
    try:
        if rain >= 10:
            base_mood = "Rainstorm Feels"
            base_query = "Arijit, Sid Sriram, Ambient"
        elif 6 <= rain < 10:
            base_mood = "Monsoon Romance"
            base_query = "Hindi + Tamil Lofi"
        elif 3 <= rain < 6 and temp <= 27:
            base_mood = "Cozy Moods"
            base_query = "Acoustic + Love Ballads"
        elif rain < 3 and temp > 30:
            base_mood = "Tropical Mood"
            base_query = "Bollywood Summer + Punjabi Beats"
        elif rain < 3 and temp >= 29 and wind < 12:
            base_mood = "Roadtrip Energy"
            base_query = "Hindi Pop + Tollywood Beats"
        elif wind >= 18:
            base_mood = "Chill Travel"
            base_query = "Hindi Chillstep, Lounge Jazz"
        elif temp < 24:
            base_mood = "Warmth & Melody"
            base_query = "Kannada, Tamil Piano, Soft Pop"
        else:
            base_mood = "Mixed Vibes"
            base_query = "Hindi Retro + English Classics"
    except:
        base_mood = "Unknown"
        base_query = "Relaxing Bollywood"

    # First full combination attempt
    base_combo = f"{base_mood} – {base_query}"
    if base_query not in used_queries:
        used_queries.add(base_query)
        return base_combo

    # Try alternative queries with same mood
    alt_queries = [
        "Kannada Classics",
        "Telugu Chill",
        "Indie Pop Vibes",
        "Soft Instrumentals",
        "English Vintage"
    ]
    for alt in alt_queries:
        if alt not in used_queries:
            used_queries.add(alt)
            return f"{base_mood} – {alt}"

    # Try fallback combinations
    for fallback_mood, fallback_query in fallback_moods:
        if fallback_query not in used_queries:
            used_queries.add(fallback_query)
            return f"{fallback_mood} – {fallback_query}"

    # Last resort: force unique name
    final_query = f"UNIQUE-{len(used_queries)}"
    used_queries.add(final_query)
    return f"{base_mood} – {final_query}"


In [67]:
df_weather_route["Playlist Mood"] = df_weather_route.apply(
    lambda row: map_weather_to_playlist(
        row["Max Temp (°C)"],
        row["Precipitation (mm)"],
        row["Wind (km/h)"],
        row["UV Index"] if row["UV Index"] is not None else 0,
        row["Location"]
    ),
    axis=1
)


In [62]:
pip install folium

## Search Tracks Using Spotify API

In [28]:
pip install spotipy


Note: you may need to restart the kernel to use updated packages.


In [30]:
def get_top_tracks_for_mood(mood_query, limit=25):
    try:
        results = sp.search(q=mood_query, type='playlist', limit=1)
        playlist_id = results['playlists']['items'][0]['id']

        tracks = sp.playlist_items(playlist_id, limit=limit)
        track_names = [
            f"{item['track']['name']} – {item['track']['artists'][0]['name']}"
            for item in tracks['items'] if item['track'] is not None
        ]
        return track_names
    except Exception as e:
        return [f"Error: {str(e)}"]


In [31]:
df_weather_route["Top Tracks"] = df_weather_route["Playlist Mood"].apply(
    lambda mood: get_top_tracks_for_mood(mood.split("–")[-1].strip())
)


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at:

## Compile Playlist for Each Route Point

In [33]:
playlist_entries = []
used_tracks = set()
track_cache = {}

for _, row in df_weather_route.iterrows():
    mood = row["Playlist Mood"]
    location = row["Location"]
    subqueries = mood.split("–")[-1].split(",")

    for subquery in subqueries:
        query = subquery.strip()

        # Cache search results
        if query not in track_cache:
            track_cache[query] = get_top_tracks_for_mood(query, limit=13)

        tracks = track_cache[query]
        print(f" Querying Spotify for: '{query}' at {location}")
        print(f"Tracks for query '{query}':\n", tracks)

        if not tracks or isinstance(tracks, str):
            print(f" No valid tracks returned for query: {query}")
            continue

        for track in tracks:
            try:
                name, artist = track.split(" – ")
                name, artist = name.strip(), artist.strip()
                track_id = (name, artist)  # unique identifier

                if track_id in used_tracks:
                    continue  # skip duplicates

                used_tracks.add(track_id)  # mark as used

                playlist_entries.append({
                    "Location": location,
                    "Mood": mood,
                    "Track Name": name,
                    "Artist": artist,
                    "Duration (min)": 3.5,
                    "Spotify Link": f"https://open.spotify.com/search/{name.replace(' ', '+')}"
                })

            except Exception as e:
                print(f" Skipping track due to error: {e}")
                continue

print(f"\n Total unique tracks collected: {len(playlist_entries)}")


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


 Querying Spotify for: 'Hindi Retro + English Classics' at Vittal Mallya Road
Tracks for query 'Hindi Retro + English Classics':
 ['Aa Chal Ke Tujhe – Kishore Kumar', 'Aap Ki Nazron Ne Samjha – Lata Mangeshkar', 'Tere Mere Sapne Ab Ek Rang Hain – Mohammed Rafi', 'Abhi Na Jao Chhod Kar – Asha Bhosle', 'Tere Bina Zindagi Se (with Narration) – Lata Mangeshkar', 'Is Mod Se Jate Hain - Duet – Lata Mangeshkar', 'Tum Aa Gaye Ho Noor Aa Gaya – Lata Mangeshkar', 'Ek Ajnabee Haseena Se – Kishore Kumar', 'Bahon Mein Chale Aao – Lata Mangeshkar', 'Kahin Door Jab Din Dhal Jaye – Mukesh', 'Pal Pal Dil Ke Paas - From "Blackmail" – Kishore Kumar', 'Jab Deep Jale Aana – K. J. Yesudas', 'Aap Ki Ankhon Mein Kuch – Kishore Kumar']


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


 Querying Spotify for: 'Bollywood Summer + Punjabi Beats' at Bangalore-Mysore Road
Tracks for query 'Bollywood Summer + Punjabi Beats':
 ['SHE IS STILL MY BABE – Romeo', 'Qayamat (From "Housefull 5") – White Noise Collectives', 'FEEL THE NIGHT – Romeo', 'Ikk Vaari - From "Mere Husband Ki Biwi" – Tanishk Bagchi', 'Sifarishan – Amar Emaan', 'Kissik (From "Pushpa 2 The Rule") [HINDI] – Lothika', 'Mafia – Harjas dhaliwal', 'Uyi Amma - From "Azaad" – Amit Trivedi', 'Djyaan Te Paaiye Bhangra – Armaan Sabharwal', 'Peelings (From "Pushpa 2 The Rule") [HINDI] – Javed Ali', 'Nain Matakka - From "Baby John" – Thaman S', 'Bhool Bhulaiyaa 3 - Title Track (feat. Pitbull) – Pitbull', "CHASIN' YOU – DANNY DIORR"]


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


...

 Querying Spotify for: 'Indie Pop Vibes' at NH275
Tracks for query 'Indie Pop Vibes':
 ['One Kiss (with Dua Lipa) – Calvin Harris', 'Diet Pepsi – Addison Rae', 'Something Just Like This – The Chainsmokers', 'Outside (feat. Ellie Goulding) – Calvin Harris', 'Rock Your Body – Justin Timberlake', 'Summer – Calvin Harris', 'ANXIETY (feat. Doechii) – Sleepy Hallow', 'Time to Pretend – MGMT', 'BMF – SZA', 'OTW – Khalid', 'Electric Feel – MGMT', "Let's Groove – Earth, Wind & Fire", 'Get Lucky (feat. Pharrell Williams and Nile Rodgers) – Daft Punk']


Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache


 Querying Spotify for: 'Soft Instrumentals' at NH275
Tracks for query 'Soft Instrumentals':
 ['deep love of Jesus - Instrumental – draw close', 'Redeemed – Khamir Music', 'pure – draw close', 'How Great Thou Art – Rasmus H Thomsen', 'Jesus We Love You – draw close', 'you are not alone – Khamir Music', 'My Savior – draw close', 'In Your Hands – Khamir Music', 'Yet - Instrumental – draw close', 'Gratitude (Peaceful Piano) – David Lindner', 'Abba - Instrumental – Christopher Galovan', 'meadows - Instrumental – Khamir Music', 'You found me - Instrumental – draw close']


Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


 Querying Spotify for: 'English Vintage' at Mysore Road
Tracks for query 'English Vintage':
 ['A Sunday Kind Of Love – Etta James', "I'll Be Seeing You – Billie Holiday", 'Dream A Little Dream Of Me – Louis Armstrong', 'Misty - Live At Deutschlandhalle, Berlin, 1960 – Ella Fitzgerald', '(I Love You) For Sentimental Reasons – Nat King Cole', 'Blue Moon – Billie Holiday', 'I Fall In Love Too Easily – Chet Baker', "Somethin' Stupid – Frank Sinatra", 'Cheek To Cheek – Ella Fitzgerald', 'Everybody Loves Somebody – Dean Martin', 'L-O-V-E – Nat King Cole', 'More - Remastered 2004 – Bobby Darin', "I'm In The Mood For Love – Julie London"]
 Querying Spotify for: 'Fusion Hindi Tamil' at Ganapathi Street
Tracks for query 'Fusion Hindi Tamil':
 ['Enthavo – Job Kurian', 'Mumbae Bounce – Sanjoy', 'N.R.I. – Raja Kumari', 'Vainko – Brodha V', 'Live It – SIRI', 'Ninna Guriya – MC Suhas', 'Namma Street – Tre Ess', ' – ', 'Meera – Raja Kumari', 'Mere Gully Mein (feat. Naezy) – DIVINE', 'Goriye – Desi Ma'

In [77]:
# Create map centered at start location
map_center = [df_weather_route["Latitude"].mean(), df_weather_route["Longitude"].mean()]
route_map = folium.Map(location=map_center, zoom_start=7)

# Add markers with mood info
marker_cluster = MarkerCluster().add_to(route_map)

for _, row in df_weather_route.iterrows():
    location = [row["Latitude"], row["Longitude"]]
    mood = row["Playlist Mood"]
    popup_info = f"""
    <b>{row['Location']}</b><br>
    Mood: {mood}<br>
    Max Temp: {row['Max Temp (°C)']}°C<br>
    Rain: {row['Precipitation (mm)']} mm<br>
    <a href="{row['Spotify Link']}" target="https://open.spotify.com/playlist/41KNDdhZ2yUZms9U5JqYXq">Spotify Search</a>
    """
    folium.Marker(location=location, popup=popup_info, icon=folium.Icon(color="blue")).add_to(marker_cluster)

route_map.save("interactive_route_map.html")
route_map  


## Create Final Playlist

In [78]:
# Auth setup 
auth_manager = SpotifyOAuth(
    client_id="ENTER-YOUR-CLIENT-ID",
    client_secret="ENTER-YOUR-CLIENT-SECRET",
    redirect_uri="http://127.0.0.1:8888/callback",  # This must match Spotify Dashboard
    scope="playlist-modify-public",
    open_browser=True,
    show_dialog=True,
)

# Print the URL and manually authorize
auth_url = auth_manager.get_authorize_url()
print(f" Please go to this URL and authorize:\n{auth_url}")

# Paste the full redirect URL you get after login
redirect_response = input(" Paste the full redirect URL here:\n")

# Get token from code in that URL
code = auth_manager.parse_response_code(redirect_response)
token_info = auth_manager.get_access_token(code)

# Create Spotify client
sp = spotipy.Spotify(auth=token_info["access_token"])

# Verify
user = sp.current_user()
print(f" Authenticated as: {user['display_name']} (ID: {user['id']})")



 Please go to this URL and authorize:
https://accounts.spotify.com/authorize?client_id=076219f7deb24b8c80f6d563a8d5f71f&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8888%2Fcallback&scope=playlist-modify-public&show_dialog=True


 Paste the full redirect URL here:
 http://127.0.0.1:8888/callback?code=AQAfJjc0KqXqKblSZScxJ_8jbGj5OO9PmKQpwx9goyhPMFbgfw3ifK8T2gjCwVExG8qma6_VKpp8xnyZLmmTzbPmX92FF9sbNG9eBHRILxcKVxFQpacydvMNwubG-3SBUugG5suROpJj-ZGwQyXGE2HFGfggm3cdon2e3AfFvKebWhOxC7pKuggMjojyi8nX_OfhGiQId5J5ng


C:\Users\crpra\AppData\Local\Temp\ipykernel_14040\3902310482.py:23: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = auth_manager.get_access_token(code)
Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


 Authenticated as: Pranav (ID: 31phr65rgsk6ubzszhfwvbqlk7bu)


In [79]:
# Playlist name and description
playlist_name = f"Tunes for the Tarmac – 05/24/2025"
playlist_description = "A travel-inspired playlist generated by location and mood. Auto-curated by code"

# Create the playlist
playlist = sp.user_playlist_create(
    user=user['id'],
    name=playlist_name,
    public=True,
    description=playlist_description
)

print("Playlist created:", playlist['external_urls']['spotify'])


Playlist created: https://open.spotify.com/playlist/1A4AGa0MQ2CWqcfHObAwKs


In [80]:
track_uris_to_add = []

for entry in playlist_entries:
    query = f"track:{entry['Track Name']} artist:{entry['Artist']}"
    results = sp.search(q=query, type='track', limit=1)

    try:
        track_uri = results['tracks']['items'][0]['uri']
        track_uris_to_add.append(track_uri)
    except IndexError:
        print(f" Not found on Spotify: {entry['Track Name']} by {entry['Artist']}")

# Split into batches of 100 (Spotify’s limit)
for i in range(0, len(track_uris_to_add), 100):
    sp.playlist_add_items(playlist_id=playlist['id'], items=track_uris_to_add[i:i + 100])

print(f" Added {len(track_uris_to_add)} tracks to playlist.")


 Not found on Spotify: Is Mod Se Jate Hain - Duet by Lata Mangeshkar
 Not found on Spotify: Kannodu Kanbadhellam - Jeans Darloo Flip Mix by Darloo
 Not found on Spotify: Aaneya Mele by Rajkumar
 Not found on Spotify: AATAVENO by Rajkumar
 Not found on Spotify: Adhey Kannu by Rajkumar
 Not found on Spotify: ALLAH ALLAH NEENE YELLA by Rajkumar
 Not found on Spotify: Kalaavathi (From "Sarkaru Vaari Paata") by Sid Sriram
 Added 135 tracks to playlist.


In [81]:
print("Your playlist is ready at:", playlist['external_urls']['spotify'])


Your playlist is ready at: https://open.spotify.com/playlist/1A4AGa0MQ2CWqcfHObAwKs


## Generate Google Maps Route Link

In [60]:
locations = [
    "Vittal Mallya Road",
    "Mysore Rd, Bengaluru, Karnataka",
    "Bengaluru - Mysuru Expressway",
    "NH 275, Karnataka",
    "Madikeri"
]

# Convert each location into a URL-safe format
from urllib.parse import quote_plus

waypoints_encoded = [quote_plus(loc) for loc in locations]

# Join all points with '+' and insert into the Google Maps direction format
maps_url = f"https://www.google.com/maps/dir/{'/'.join(waypoints_encoded)}"

print("Google Maps Route Link:")
print(maps_url)


Google Maps Route Link:
https://www.google.com/maps/dir/Vittal+Mallya+Road/Mysore+Rd%2C+Bengaluru%2C+Karnataka/Bengaluru+-+Mysuru+Expressway/NH+275%2C+Karnataka/Madikeri


In [75]:
route_coords = df_weather_route[["Latitude", "Longitude"]].values.tolist()

animated_map = folium.Map(location=map_center, zoom_start=7)
AntPath(route_coords, color="green", weight=5, delay=800).add_to(animated_map)

# Add start and end
folium.Marker(route_coords[0], tooltip="Start", icon=folium.Icon(color="green")).add_to(animated_map)
folium.Marker(route_coords[-1], tooltip="End", icon=folium.Icon(color="red")).add_to(animated_map)

animated_map.save("animated_route.html")
animated_map
